In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## ram
import gzip
import gc #garbage collector

# Surprise
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import cross_validate

In [52]:
## function para leer cada linea de los datos comprimidos
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [53]:
#### Importar los datos de las caracteristicas de los juegos

## Funcion que da acamica para abrir solo cada cierta cantidad de datos.
'''
contador = 0
data_reviews = []
# Vamos a guardar una de cada 10 reviews para no llenar la memoria RAM. Si pones n = 3, 
# abrira uno de cada tres, y asi.
n = 10
for l in parse('steam_reviews.json.gz'):
    if contador%n == 0:
        data_reviews.append(l)
    else:
        pass
    contador += 1
'''
## Funcion para coger solo una cantidad de datos en orden
data_games = []
num = 1000
for i, l in enumerate(parse('datos\steam_games.json.gz')):
    data_games.append(l)
    if i == num:
        break

In [54]:
#### Importar los datos de las reviews

## Funcion que da acamica para abrir solo cada cierta cantidad de datos.
'''
contador = 0
data_reviews = []
# Vamos a guardar una de cada 10 reviews para no llenar la memoria RAM. Si pones n = 3, 
# abrira uno de cada tres, y asi.
n = 10
for l in parse('steam_reviews.json.gz'):
    if contador%n == 0:
        data_reviews.append(l)
    else:
        pass
    contador += 1
'''
## Funcion para coger solo una cantidad de datos en orden
data_reviews = []
num = 1000
for i, l in enumerate(parse('datos\steam_reviews.json.gz')):
    data_reviews.append(l)
    if i == num:
        break

In [62]:
## Crear un data frame con los datos
df_games = pd.DataFrame(data_games)
df_rev = pd.DataFrame(data_reviews)
# df_rev.head()
# df_games.head()

# Analisis exploratorio de datos

In [86]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       974 non-null    object 
 1   genres          961 non-null    object 
 2   app_name        1000 non-null   object 
 3   title           990 non-null    object 
 4   url             1001 non-null   object 
 5   release_date    989 non-null    object 
 6   tags            986 non-null    object 
 7   discount_price  16 non-null     float64
 8   reviews_url     1000 non-null   object 
 9   specs           997 non-null    object 
 10  price           957 non-null    object 
 11  early_access    1001 non-null   bool   
 12  id              1000 non-null   object 
 13  developer       977 non-null    object 
 14  sentiment       905 non-null    object 
 15  metascore       406 non-null    object 
dtypes: bool(1), float64(1), object(14)
memory usage: 118.4+ KB


## Analisis de datos de info games
- Se ve que las variables de discount_price y las de metascore faltan mucho
- Se observa que esta la feature "sentiment" que nos dice el rating general de el juego
- El nombre del juego es la feature 'title' pero tambien "app name" se puede usar como substituto
- los atributos generos y tags son muy parecidos entre si, solo que genero es solamente tiene los generos de los juegos, pero por esta razon tiene mas naans que tags
- Hay caracteres no occidentales y emojis asi que hay que tener cuidado con los errores de encoding

In [98]:
df_rev.info()
df_rev.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   username      1001 non-null   object 
 1   hours         1000 non-null   float64
 2   products      1001 non-null   int64  
 3   product_id    1001 non-null   object 
 4   page_order    1001 non-null   int64  
 5   date          1001 non-null   object 
 6   text          1001 non-null   object 
 7   early_access  1001 non-null   bool   
 8   page          1001 non-null   int64  
 9   found_funny   207 non-null    float64
 10  compensation  61 non-null     object 
 11  user_id       398 non-null    object 
dtypes: bool(1), float64(2), int64(3), object(6)
memory usage: 87.1+ KB


,username,hours,products,product_id,page_order,date,text,early_access,page,found_funny,compensation,user_id
0,Chaos Syren,0.1,41,725280,0,2017-12-17,This would not be acceptable as an entertainme...,False,1,NaN,NaN,NaN
1,₮ʜᴇ Wᴀʀᴛᴏɴ,51.1,769,328100,0,2017-12-27,looks like a facebook game,False,1,NaN,NaN,NaN
2,hello?<,14.6,2,328100,1,2017-10-16,Better than Minecraft,False,1,2.0,Product received for free,NaN
3,Cyderine916,5.0,64,35140,0,2018-01-04,I love and idolized Batman and this game is Ma...,False,1,NaN,NaN,NaN
4,DarklyThinking,16.6,577,35140,1,2018-01-04,Still worth playing in 2018.\nProbably my favo...,False,1,NaN,NaN,76561198007483075


In [105]:
len(df_rev[df_rev.duplicated(subset=['username'],keep=False)])

27

# Analisis de Features datos de review
- hay que usar los datos de username para identificar a los usuarios (creo que los nombres son unicos). Hay que ver de alguna manera si dos personas pueden tener el mismo username.
- El 2.7 porciento del dataset es de personas que hacen varias reviews (Hacer graficos con el numero de reviews que hace una persona)
- La feature "text" puede servir para hacer un analisis de sentimiento
- El numero de horas jugadas puede encodearse para utilizarlo como un suplente de rating

# Graficos
- Coger mas datos a la manera de acamica. 
Para hacer estos graficos necesitamos una dataset mas grande y representativo. Tendremos que usar la funcion que lee una row del dataset cada cierto numero de lines(la funcion que nos da acamica) para evitar solo coger datos de cierto tipo de juegos. Como las reviews estan en un orden, al usar la funcion de datos podriamo accidentalmente, y si las suerte no nos acompaña, escoger un tipo de juego que no represente la totalidad de los juegos, por ejemplog, que en los 1000 datos ordenas cojamos solo los juegod free-to-play, o los juegos indies, que posiblemente tengan reviews mas bajas que otros..


- Hacer un grafico de las horas jugadas del juego #todo
- Hacer un grafico de el numero de reviews que hace la gente #todo

## PARA HACER
- Podemos usar el target como la combinacion de el numero de horas jugadas combinado con el analisis de sentimiento de la review. (primero hacer con el encoding ) #todo